# Proyecto Final

***Equipo 07***

- Aide Jazmín González Cruz
- Elena Villalobos Nolasco
- Carolina Acosta Tovany

#### Instrucciones

El proyecto/examen final consistirá en:

La implementación del algoritmo de filtrado colaborativo utilizando la metodología vista en clase (uso de otra metodología no se calificará).

Todos los algoritmos de aprendizaje de máquina que se utilicen deberán haber sido creados por ustedes. Sólo podrán utilizar Transformers y funciones de apoyo de scikit-learn (para realizar la división de los datos en entrenamiento y prueba, o el procedimiento de validación cruzada, etc.) mas ningún estimator (regresión logística, máquina de vectores de soporte, k medias, etc.). 

Se deberá explicar como se obtuvo la k con la que se generó el resultado final.

Se utilizarán los archivos con el conjunto pequeño de calificaciones y películas ubicado en la siguiente https://www.kaggle.com/rounakbanik/the-movies-dataset:

- **links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

- **ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

Con el fin de mejorar la calificación (opcional, puntos extra), se podrán utilizar los algoritmos desarrollado en las tareas del curso y los datos relevantes (los que hacen match con los datos anteriores) contenidos en los archivos:

- **movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

- **keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

- **credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

La métrica con la que se determinará el desempeño del algoritmo es el NDCG 

(https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG)

Una vez obtenida la matriz de calificaciones, el programa deberá ser capaz de regresar las 5 mejores recomendaciones del o de los usuarios que se consulten.

El proyecto se entregará en un Jupyter notebook. El readme file debe contener las instrucciones para que se ejecute el código. Deben cerciorarse que siguiendo esas instrucciones el programa corre sin errores. 

Se deberá subir a la carpeta proyecto_final/equipo_xx en el repositorio GitHub antes de las 7:00 am del día del examen final (14 de diciembre de 2020).    

In [1]:
# Importación de paqueterías necesarias
import pandas as pd
import numpy as np
import random

La función objetivo:
    
$$J(X) = \frac{1}{2} \displaystyle\sum_{(a,i)\in\mathbb{D}} \left(Y_{ai}-\left [ UV^T \right ]_{ai} \right)^2 + \frac{\lambda}{2} \displaystyle\sum_{a=1}^n \displaystyle\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k V_{ij}^2$$

In [2]:
def load_data():
    """
    Carga datos
    Regresa dataframe de Usuarios, Películas y raitings
    """
    
    # Carga de datos
    links_small = pd.read_csv('links_small.csv')
    ratings_small = pd.read_csv('ratings_small.csv')
    
    # Películas en catálogo que no han calificado los usuarios
    df_mov_u = pd.DataFrame(ratings_small['movieId'])
    df_mov = pd.DataFrame(links_small['movieId'])

    common = df_mov.merge(df_mov_u, on=["movieId"])
    result = df_mov[~df_mov.movieId.isin(common.movieId)]
    
    # Construyendo la matriz Y_ai
    y_ia = links_small.set_index('movieId').join(ratings_small.set_index('movieId'))
    y_ia = y_ia.reset_index()
    #y_ia.pivot(index="userId", columns="movieId", values="rating") 
    y_ia = pd.DataFrame(y_ia.pivot(index='userId', columns='movieId', values='rating'))
    y_ia = pd.DataFrame(y_ia.to_records())
    # Eliminando usuario Nan
    y_ia = y_ia[pd.notnull(y_ia['userId'])]
    # Borrando columna 1 con user_id
    y_ia = y_ia.drop(['userId'], axis=1)
    # Cambiando Nan por zeros
    #y_ia[np.isnan(y_ia)] = 0
    
    return y_ia
    

In [3]:
Y = load_data()
Y

FileNotFoundError: [Errno 2] File links_small.csv does not exist: 'links_small.csv'

In [37]:
maxValues = Y.max() 
maxValues = pd.DataFrame(maxValues)
maxValues = maxValues.max() 
maxValues

0    5.0
dtype: float64

In [39]:
minValues = Y.min() 
minValues = pd.DataFrame(minValues)
minValues = minValues.min() 
minValues

0    0.5
dtype: float64

In [85]:
# Cambiando Nan por zeros
Y_0 = Y.copy()
Y_0[np.isnan(Y_0)] = 0
Y_0

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
670,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# Cambiando Nan a boleanos
Y.isna()

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
5,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
668,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
669,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
670,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [87]:
Prueba = Y.iloc[[0,1,2,3,4], : 10]
Prueba

,1,2,3,4,5,6,7,8,9,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
Prueba.iloc[1][1] = 1
Prueba.iloc[1][4] = 5
Prueba

,1,2,3,4,5,6,7,8,9,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# Fijando semilla
random.seed(0)
random_matrix = np.random.randint(1,5,(5,10))
random_matrix = pd.DataFrame(random_matrix)
random_matrix

,0,1,2,3,4,5,6,7,8,9
0,3,2,3,1,1,1,4,3,2,4
1,2,2,4,2,3,3,2,4,2,2
2,2,1,1,4,4,4,2,3,2,1
3,1,1,3,3,2,4,2,3,4,2
4,3,3,2,3,4,4,2,1,1,4


In [93]:
Prueba_bool = Prueba.isna()
Prueba_bool

,1,2,3,4,5,6,7,8,9,10
1,True,True,True,True,True,True,True,True,True,True
2,True,False,True,True,False,True,True,True,True,False
3,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,False
5,True,True,False,True,True,True,True,True,True,True


In [104]:
random_matrix.iloc[1][1]*Prueba_bool.iloc[1][1]
random_matrix.iloc[1]

0    2
1    2
2    4
3    2
4    3
5    3
6    2
7    4
8    2
9    2
Name: 1, dtype: int64

In [152]:
nvos = []
for i in range (len(random_matrix.iloc[1])):
    nvos.append([i+1,random_matrix.iloc[1][i]*Prueba_bool.iloc[1][i]])
    
nvos = pd.DataFrame(nvos)
nvos.columns = ['id_movie','rating_recom']
nvos = nvos.sort_values(by=['rating_recom'], ascending=False)
nvos

,id_movie,rating_recom
2,3,4
7,8,4
5,6,3
0,1,2
3,4,2
6,7,2
8,9,2
1,2,0
4,5,0
9,10,0


In [153]:
nvos = nvos[(nvos[['rating_recom']] != 0).all(axis=1)]
nvos

,id_movie,rating_recom
2,3,4
7,8,4
5,6,3
0,1,2
3,4,2
6,7,2
8,9,2


In [154]:
nvos['id_movie'].head(5).to_numpy()

array([3, 8, 6, 1, 4])

In [126]:
recomendaciones = nvos.iloc[[0,1,2,3,4], : 1].to_numpy()
recomendaciones

array([[3],
       [8],
       [6],
       [1],
       [4]])

In [129]:
np.concatenate(recomendaciones, axis=0 )

array([3, 8, 6, 1, 4])

In [155]:
def recomendaciones_id(Y, MCompleta, id_user, top=5):
    """
    Regresa id de las peliculas reomendadas dado un usuario
    Param: Recibe matriz con Nan, id del usurario y el top por default 5
    Return: Arreglo de ids de las películas sugeridas
    """
    # Matriz auxiliar booleana
    Y_bool = Y.isna()
    # Se multiplica la matriz boleana con Y para poner en cero los ratings dados por el usuario
    nvos = []
    #print(MCompleta.iloc[id_user-1])
    for i in range (len(MCompleta.iloc[id_user-1])):
        #Se suma 1 por que los usuarios inician en 1
        nvos.append([i+1,MCompleta.iloc[id_user-1][i]*Y_bool.iloc[id_user-1][i]])
        
    # nvos es un arreglo se pasa a dataframe para mejor manejabilidad
    nvos = pd.DataFrame(nvos)
    # se colocan nombres a las columnas
    nvos.columns = ['id_movie','rating_recom']
    # se ordenan de forma descendente
    nvos = nvos.sort_values(by=['rating_recom'], ascending=False)
    # Borrando 0
    nvos = nvos[(nvos[['rating_recom']] != 0).all(axis=1)]
    # Obteniendo el top
    recomendaciones = nvos['id_movie'].head(5).to_numpy()
    
    return recomendaciones


In [156]:
recomendaciones_id(Prueba, random_matrix, 2)

array([3, 8, 6, 1, 4])

Tenemos una $k = 1$

In [47]:
# Fijando semilla
random.seed(0)
# Creando el vector V que son las películas al azar
V = np.random.randint(1,9,size = (1,movies))
V.shape

(1, 9125)

In [48]:
# Sacando el vector U que son los usuarios
U = np.random.randint(1,9,size = (1,users))
U.shape

(1, 671)

In [2]:
from sympy import *

La función objetivo:
    
$$J(X) = \frac{1}{2} \displaystyle\sum_{(a,i)\in\mathbb{D}} \left(Y_{ai}-\left [ UV^T \right ]_{ai} \right)^2 + \frac{\lambda}{2} \displaystyle\sum_{a=1}^n \displaystyle\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k V_{ij}^2$$

#### Fórmulas a aplicar

$$x_u^T = r_uY\left(Y^TY+\lambda_xI)^{-1}\right)$$

$$y_i^T = r_iX\left(X^TX+\lambda_yI)^{-1}\right)$$

In [4]:
Y = pd.DataFrame([[5,np.NaN,7],[1,2,np.NaN]])
Y

,0,1,2
0,5,NaN,7.0
1,1,2.0,NaN


In [5]:
r,c = Y.shape
print("rows: ", r, " cols: ", c)

rows:  2  cols:  3


In [6]:
# Para manejar NaN Values
Y_0 = Y.notna()
Y_0

,0,1,2
0,True,False,True
1,True,True,False


In [8]:
# Para manejar NaN Values
Yia = Y.copy()
Yia[np.isnan(Yia)] = 0
Yia

,0,1,2
0,5,0.0,7.0
1,1,2.0,0.0


In [59]:
def merge_list_f(lists):
    merged_list = []
    for l in lists:
        merged_list += l
    return merged_list

In [56]:
# Borrando 0
Aux = [[1],[2],[3]]
Aux = merge_list_f(Aux)
Aux

array([1, 2, 3])

$$x_u^T = r_uY\left(Y^TY+\lambda_xI)^{-1}\right)$$

### Extendiendo funcionalidad para k

In [ ]:
# Fijando semilla
# random.seed(0) -> noffunciona
# DESCOMENTAR CUANDO SEA RANDOM
# Creando el vector V que son las películas al azar
k = 1
#V = np.random.randint(1, 9, size=(k, c))
#pd.DataFrame(V)

In [92]:
# Para k = 1
# Para esta prueba
Vi = pd.DataFrame([[2,7,8]])
Vi

,0,1,2
0,2,7,8


In [12]:
#CREANDO VECTORES SIN NaN para k = 1
R = []
Yu = []
Aux = []
Aux_v = [] 
k, c = Vi.shape
Xu = []

print("k:",k)
# Fijando V (Y)
for row in range(r):
    for col in range(c):
        #print("V: ",Vaux.iloc[row, col])
        if ((Yia.iloc[row, col]) != 0):
            #print("Y: ",Y.iloc[row, col])
            Aux.append(Y.iloc[row, col]) 
            print("col: ", col)
            print("V[col]: ", Vi[col].tolist())
            Aux_v.append(Vi[col].tolist())
            print("Aux_v: ", Aux_v)
    
    print("Aux: ",Aux)
    R = Aux
    if(k==1):
        Aux_v = merge_list_f(Aux_v)
    print("Aux_v: ", Aux_v)   
    Yu = np.array(Aux_v) 
    print("Yu: ",Yu)
    print("------")
    Aux = []
    Aux_v = []
    
    # Comenzamos fórmula
    RY = R*Yu.T
    RY = sum(RY)
    print("RY: ", RY)
    YTY = Yu.T*Yu
    YTY = sum(YTY)
    print("YTY: ", YTY)
    LamI = 1
    print("YTY+LamI: ", YTY+LamI)
    Xu.append(RY*(1/(YTY+LamI)))
    print("Xu: ",Xu)

k: 1
col:  0
V[col]:  [2]
Aux_v:  [[2]]
col:  2
V[col]:  [8]
Aux_v:  [[2], [8]]
Aux:  [5, 7.0]
Aux_v:  [2, 8]
Yu:  [2 8]
------
RY:  66.0
YTY:  68
YTY+LamI:  69
Xu:  [0.9565217391304348]
col:  0
V[col]:  [2]
Aux_v:  [[2]]
col:  1
V[col]:  [7]
Aux_v:  [[2], [7]]
Aux:  [1, 2.0]
Aux_v:  [2, 7]
Yu:  [2 7]
------
RY:  16.0
YTY:  53
YTY+LamI:  54
Xu:  [0.9565217391304348, 0.2962962962962963]


In [88]:
# Para k = 2
# Para esta prueba
Vi = pd.DataFrame([[1,2,3],[4,5,6]])
Vi

,0,1,2
0,1,2,3
1,4,5,6


#### Calculo de V

In [93]:
# Obteniendo vector U (fijando V)

#CREANDO VECTORES SIN NaN para k = 1,2
R = []
Yu = []

# Guarda valores de Y que no son cero
Rui = []

# Guarda valores de V que no son cero en la posición de Y
Aux_v = [] 

k, c = Vi.shape
Xu = []

print("k:",k)
# Fijando V (Y)

r,c = Y.shape
#r = 1

# Lambda
Lambda = 1
# Identidad dpende de k
I = np.identity(k)

# Recorre filas de la matriz UV -> Y
for row in range(r):
    
    # Recorre columnas de la matriz UV -> Y
    for col in range(c):
        
        # Checa ceros (NaN), si no es 0 ejecuta operación
        if ((Yia.iloc[row, col]) != 0):
            #print("Y: ",Y.iloc[row, col])
            # Toma el elemnto de la matriz Y
            Rui.append(Y.iloc[row, col]) 
            print("V[col]: ", Vi[col].tolist())
            # Toma la columna de V
            Aux_v.append(Vi[col].tolist())
            #print("Aux_v: ", Aux_v)
    
    print("Rui: ",Rui)
    R = np.array(Rui)
    print("R: ",R)
    if(k==1):
        Aux_v = merge_list_f(Aux_v)
        I = I[0]
        
    print("Aux_v: ", Aux_v)   
    Yu = np.array(Aux_v).T
    print("Yu: ",Yu)
    Rui = []
    Aux_v = []
        
    # Comenzamos fórmula
    RY = np.matmul(R,Yu)#.dot
    print("RY: ", RY)
    YTY = np.matmul(Yu,Yu.T)
    print("YTY: ", YTY)
    LamI = Lambda*I
    if(k==1):
        Xu.append(RY*(1/(YTY+LamI)))
    else:
        Minv = YTY+LamI
        print("YTY+LamI: ", Minv)
        print("YTY+LamI-1: ",np.linalg.inv(Minv))
        Xu.append(np.matmul(RY,np.linalg.inv(YTY+LamI)))
    print("Xu: ",Xu)
    print("Frac: ",RY,"/",YTY+LamI)
    print("-------------------------------------------------------")
    print(np.array(Xu))


k: 1
V[col]:  [2]
V[col]:  [8]
Rui:  [5, 7.0]
R:  [5. 7.]
Aux_v:  [2, 8]
Yu:  [2 8]
RY:  66.0
YTY:  68
Xu:  [array([0.95652174])]
Frac:  66.0 / [69.]
-------------------------------------------------------
[[0.95652174]]
V[col]:  [2]
V[col]:  [7]
Rui:  [1, 2.0]
R:  [1. 2.]
Aux_v:  [2, 7]
Yu:  [2 7]
RY:  16.0
YTY:  53
Xu:  [array([0.95652174]), 0.2962962962962963]
Frac:  16.0 / 54.0
-------------------------------------------------------
[array([0.95652174]) 0.2962962962962963]


#### Calculo de U

$$y_i^T = r_iX\left(X^TX+\lambda_yI)^{-1}\right)$$

In [ ]:
# Obteniendo vector V (fijando U)

#CREANDO VECTORES SIN NaN para k = 1,2
R = []
Yu = []

# Guarda valores de Y que no son cero
Rui = []

# Guarda valores de V que no son cero en la posición de Y
Aux_v = [] 

k, c = Vi.shape
Xu = []

print("k:",k)
# Fijando V (Y)

r,c = Y.shape
#r = 1

# Lambda
Lambda = 1
# Identidad dpende de k
I = np.identity(k)

# Columns
for j in range(c):
    e = 0
    # Rows
    for i in range(r):    
        print("Y ",Y[i,j])
        print("E ",Exp[i,j])
        print("YE ",Y[i,j]-Exp[i,j])
        if ((Y[i,j]) != 0):
            e = e + ((Y[i,j]-Exp[i,j])**2)/2
            
        print("e: ",e)
        
    e = e + (v**2)/2     
    e
    e = derive_by_array(e, v)
    e
    h = solve(e,v)
    Vaux.append(h)
    print("sol ",h)
    
print(Vaux)


# Recorre columnas de la matriz UV -> Y
for col in range(c):

    # Recorre filas de la matriz UV -> Y
    for row in range(r):    
        
        # Checa ceros (NaN), si no es 0 ejecuta operación
        if ((Yia.iloc[row, col]) != 0):
            #print("Y: ",Y.iloc[row, col])
            # Toma el elemento de la matriz Y
            Rui.append(Y.iloc[row, col]) 
            print("V[col]: ", Vi[col].tolist())
            # Toma la columna de V
            Aux_v.append(Vi[col].tolist())
            #print("Aux_v: ", Aux_v)
    
    print("Rui: ",Rui)
    R = np.array(Rui)
    print("R: ",R)
    if(k==1):
        Aux_v = merge_list_f(Aux_v)
        I = I[0]
        
    print("Aux_v: ", Aux_v)   
    Yu = np.array(Aux_v).T
    print("Yu: ",Yu)
    Rui = []
    Aux_v = []
        
    # Comenzamos fórmula
    RY = np.matmul(R,Yu)#.dot
    print("RY: ", RY)
    YTY = np.matmul(Yu,Yu.T)
    print("YTY: ", YTY)
    LamI = Lambda*I
    if(k==1):
        Xu.append(RY*(1/(YTY+LamI)))
    else:
        Minv = YTY+LamI
        print("YTY+LamI: ", Minv)
        print("YTY+LamI-1: ",np.linalg.inv(Minv))
        Xu.append(np.matmul(RY,np.linalg.inv(YTY+LamI)))
    print("Xu: ",Xu)
    print("Frac: ",RY,"/",YTY+LamI)
    print("-------------------------------------------------------")